# 1. Importing the useful libraries

In [20]:
# Importation of useful libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [21]:
basket = pd.read_csv('basket.csv')
basket

,CustomerID,StockCode
0,12346,['23166']
1,12347,"['17021', '85178', '21976', '23175', '84559B',..."
2,12348,"['23078', '84991', '84988', '23076', '21211', ..."
3,12349,"['23113', '20685', '23108', '22553', '35970', ..."
4,12350,"['22620', '20652', '21915', '22348', '20615', ..."
...,...,...
4329,18280,"['22084', '22467', '22727', '22495', '82484', ..."
4330,18281,"['22028', '22467', '22716', '23007', '23209', ..."
4331,18282,"['22699', '21108', '23295', '23187', '21270', ..."
4332,18283,"['23551', '23581', '21890', '21929', '22367', ..."


#### Remmove Punctuation in StockCode

In [22]:
basket['StockCode'] = basket['StockCode'].str.replace(r'[^\w\s]+', '')
basket

C:\Users\Admins\AppData\Local\Temp/ipykernel_10412/2075465518.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  basket['StockCode'] = basket['StockCode'].str.replace(r'[^\w\s]+', '')


,CustomerID,StockCode
0,12346,23166
1,12347,17021 85178 21976 23175 84559B 84997B 22697 23...
2,12348,23078 84991 84988 23076 21211 23077 21982 2295...
3,12349,23113 20685 23108 22553 35970 23295 37500 2272...
4,12350,22620 20652 21915 22348 20615 21866 79191C 225...
...,...,...
4329,18280,22084 22467 22727 22495 82484 22725 22358 2261...
4330,18281,22028 22467 22716 23007 23209 23008 22037
4331,18282,22699 21108 23295 23187 21270 23174 22423 2208...
4332,18283,23551 23581 21890 21929 22367 22720 22385 2329...


# 2. Create crucial function 

In [23]:
def list_rule(data):
    list_condition = []
    list_result = []
    for rule_index in range(len(data)):

        condition = []
        result = []
        for condition_index in range(len(data['Condition'][rule_index].split())):
            for result_index in range(len(data['Result'][rule_index].split())):
                condition.append(data['Condition'][rule_index].split()[condition_index])
                result.append(data['Result'][rule_index].split()[result_index])
        list_condition.append(set(condition))
        list_result.append(set(result))
    return list_condition,list_result

In [24]:
def compute_accur_each_basket(data,list_condition,list_result):
    accuracy = {sentences: [0] for sentences in range(len(data))}
    # sum = len(list_result)
    for basket_index in range(len(data)):
        list_stockcode = []

        for i in range(len(data['StockCode'][basket_index].split())):
            list_stockcode.append(data['StockCode'][basket_index].split()[i])

        count = 0 
        have_condition = 0 

        for i in range(len(list_condition)):
            if (list_condition[i]).issubset(set(list_stockcode)):
                have_condition += 1
                if (list_result[i]).issubset(set(list_stockcode)):
                    count += 1
        if (count == 0) and (have_condition == 0):
            accuracy[basket_index] = 1
        else:
            accuracy[basket_index] = count/have_condition
    return accuracy

In [25]:
def evaluate_model(accuracy):
    total_accuracy = 0
    for i in range(len(accuracy)):
        total_accuracy += accuracy[i]
    percent = total_accuracy/len(accuracy)
    print('Model accuracy ', percent*100,'%')

## Evaluate without clustering

In [26]:
rule_without_cluster = pd.read_csv("rulecluster_sum-cleaned.csv")
rule_without_cluster.head()

,Condition,Result
0,22699 22698,22697
1,22423 22699 22698,22697
2,22423 22698,22697
3,22698,22697
4,22423 22697 22698,22699


In [27]:
data_without_test = pd.read_csv("csv_result-dfm_done_cluster_test.csv")
data_without_test.head()

,id,CustomerID,10002,10080,10120,10123C,10124A,10124G,10125,10133,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
0,1,17081,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
1,2,17082,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2,3,17083,?,?,?,?,?,?,?,t,...,?,?,?,?,?,?,?,?,?,?
3,4,17084,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
4,5,17085,t,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


In [28]:
cluster2_test= data_without_test.merge(basket,how='left')
cluster2_test = cluster2_test.loc[:,['CustomerID','StockCode']]
np.shape(cluster2_test)

(867, 2)

In [29]:
list_condition,list_result = list_rule(rule_without_cluster)

In [30]:
accuracy_without = compute_accur_each_basket(cluster2_test,list_condition,list_result)

In [31]:
total_accuracy = evaluate_model(accuracy_without)
total_accuracy

Model accuracy  80.73864978072433 %


# Evaluate With Clustering

In [32]:
cluster0 = pd.read_csv('csv_result-dfm_done_cluster0_test.csv')
cluster1 = pd.read_csv('csv_result-dfm_done_cluster1_test.csv')
cluster2 = pd.read_csv('csv_result-dfm_done_cluster2_test.csv')

#### Cluster 0 

In [33]:
cluster0_test= cluster0.merge(basket,how='left')
cluster0_test = cluster0_test.loc[:,['CustomerID','StockCode']]
np.shape(cluster0_test)

(823, 2)

In [34]:
rule_cluster0 = pd.read_csv('rulecluster0-cleaned.csv')
list_condition0,list_result0 = list_rule(rule_cluster0)

In [35]:
accuracy_cluseter0 = compute_accur_each_basket(cluster0_test,list_condition0,list_result0)
total_accuracy0 = evaluate_model(accuracy_cluseter0)
total_accuracy0

Model accuracy  89.03732388847821 %


#### Cluster 1

In [36]:
cluster1_test= cluster1.merge(basket,how='left')
cluster1_test = cluster1_test.loc[:,['CustomerID','StockCode']]
np.shape(cluster1_test)

(2, 2)

In [37]:
rule_cluster1 = pd.read_csv('rulecluster1-cleaned.csv')
list_condition1,list_result1 = list_rule(rule_cluster1)

In [38]:
accuracy_cluseter1 = compute_accur_each_basket(cluster1_test,list_condition1,list_result1)
total_accuracy1 = evaluate_model(accuracy_cluseter1)
total_accuracy1

Model accuracy  91.27906976744187 %


#### Cluster 2

In [39]:
cluster2_test= cluster2.merge(basket,how='left')
cluster2_test = cluster2_test.loc[:,['CustomerID','StockCode']]
np.shape(cluster2_test)

(41, 2)

In [40]:
rule_cluster2 = pd.read_csv('rulecluster2-cleaned.csv')
list_condition2,list_result2 = list_rule(rule_cluster2)

In [41]:
accuracy_cluseter2 = compute_accur_each_basket(cluster2_test,list_condition2,list_result2)
total_accuracy2 = evaluate_model(accuracy_cluseter2)
total_accuracy2

Model accuracy  78.87915742793791 %
